# Notebook 01 — Dataset Construction from MIMIC-IV / MIMIC-IV-Note

Mục tiêu: xây dựng một tập dữ liệu gọn để huấn luyện mô hình văn bản → đa nhãn (ICD-block và nhóm xét nghiệm giai đoạn sớm), với khả năng giới hạn số dòng đọc ở mọi bước nhằm phục vụ demo nhanh và tiết kiệm tài nguyên.

Quy ước trình bày cho mỗi bước:
1) Mục đích của bước và lý do cần thiết.
2) Trường dữ liệu được sử dụng và ý nghĩa của từng trường; nếu nối giữa các bảng, chỉ rõ khóa nối và ý nghĩa của khóa.
3) Thực hiện xử lý dữ liệu.
4) Diễn giải ý nghĩa các trường trong kết quả hiển thị.

## 0) Cấu hình & nguyên tắc đọc dữ liệu (không dùng `nrows`)

**Quan trọng:** Không tự động giới hạn số dòng bằng `nrows`. Thay vào đó:
- Chọn **COHORT** ngay ở bước setup bằng biến `N_HADM` (số lần nhập viện muốn lấy).
- Ở các bước sau, **đọc theo `chunksize`** và **lọc theo `COHORT_HADM`** để đảm bảo khóa nhất quán giữa admissions/ICD/Lab/Notes.

**Tham số chính trong bước này:**
- `N_HADM` *(int | None)*: số lần nhập viện trong cohort (đặt `None` để dùng toàn bộ).
- `SEED` *(int)*: hạt giống ngẫu nhiên khi cần chọn ngẫu nhiên.
- `BALANCE_BY_SUBJECT` *(bool)*: gợi ý cân bằng theo `subject_id` khi chọn cohort.
- `CHUNKSIZE_DEFAULT` *(int)*: kích thước chunk chuẩn khi đọc các bảng lớn.
- `TOP_LABS` *(int)*: số lab phổ biến (0–6h) dùng làm vocab.

In [3]:
# ======================================================
# BƯỚC 1 — CẤU HÌNH DỮ LIỆU VÀ THAM SỐ TIỀN XỬ LÝ
# ======================================================

from pathlib import Path
import pandas as pd

# ====== Tham số cohort & đọc file ======
N_HADM = 10_000             # vd: None để dùng toàn bộ admissions
SEED = 42                   # reproducibility
BALANCE_BY_SUBJECT = True
CHUNKSIZE_DEFAULT = 500_000  # đọc file lớn theo chunk

# ====== Dò đường dẫn dữ liệu subset ======
CANDIDATES = [Path("data"), Path("../data"), Path("../../data")]
DATA_ROOT = None
for cand in CANDIDATES:
    if (cand / "mimiciv_subset").exists():
        DATA_ROOT = cand
        break
if DATA_ROOT is None:
    raise FileNotFoundError(
        "❌ Không tìm thấy thư mục 'data/mimiciv_subset'. "
        "Hãy điều chỉnh CANDIDATES hoặc thiết lập DATA_ROOT thủ công."
    )

SUBSET_DIR = DATA_ROOT / "mimiciv_subset"

# ====== Đọc số lượng thực tế trong subset ======
try:
    TOP_LABS = len(pd.read_csv(SUBSET_DIR / "labs.csv"))
    TOP_ICD = len(pd.read_csv(SUBSET_DIR / "icd.csv"))
    TOP_PROCS = len(pd.read_csv(SUBSET_DIR / "procedures.csv"))
except Exception as e:
    raise RuntimeError(f"⚠️ Lỗi khi đọc các file subset: {e}")

# ====== Cấu hình đường dẫn ======
ADMISSIONS_PATH = SUBSET_DIR / "admissions.csv"
PATIENTS_PATH   = SUBSET_DIR / "patients.csv"
DISCHARGE_PATH  = SUBSET_DIR / "discharge.csv.gz"
ICD_PATH        = SUBSET_DIR / "icd.csv"
PROCS_PATH      = SUBSET_DIR / "procedures.csv"
LABS_PATH       = SUBSET_DIR / "labs.csv"
D_ICD_DIAG_PATH = SUBSET_DIR / "d_icd_diagnoses.csv"
D_ICD_PROC_PATH = SUBSET_DIR / "d_icd_procedures.csv"

# ====== In thông tin cấu hình ======
print("✅ Cấu hình dữ liệu thành công")
print("DATA_ROOT:", DATA_ROOT.resolve())
print({
    "N_HADM": N_HADM,
    "SEED": SEED,
    "BALANCE_BY_SUBJECT": BALANCE_BY_SUBJECT,
    "CHUNKSIZE_DEFAULT": CHUNKSIZE_DEFAULT,
    "TOP_LABS": TOP_LABS,
    "TOP_ICD": TOP_ICD,
    "TOP_PROCS": TOP_PROCS,
})
print("\n📂 Các file đầu vào:")
for path in [ADMISSIONS_PATH, PATIENTS_PATH, DISCHARGE_PATH, ICD_PATH,
             PROCS_PATH, LABS_PATH, D_ICD_DIAG_PATH, D_ICD_PROC_PATH]:
    print(" -", path.relative_to(DATA_ROOT))


✅ Cấu hình dữ liệu thành công
DATA_ROOT: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data
{'N_HADM': 10000, 'SEED': 42, 'BALANCE_BY_SUBJECT': True, 'CHUNKSIZE_DEFAULT': 500000, 'TOP_LABS': 50, 'TOP_ICD': 50, 'TOP_PROCS': 50}

📂 Các file đầu vào:
 - mimiciv_subset/admissions.csv
 - mimiciv_subset/patients.csv
 - mimiciv_subset/discharge.csv.gz
 - mimiciv_subset/icd.csv
 - mimiciv_subset/procedures.csv
 - mimiciv_subset/labs.csv
 - mimiciv_subset/d_icd_diagnoses.csv
 - mimiciv_subset/d_icd_procedures.csv


### Tiện ích đọc dữ liệu & xử lý cơ bản
- `read_csv_chunks(path, usecols=None, chunksize=CHUNKSIZE_DEFAULT)`: **generator** đọc theo chunk, KHÔNG dùng `nrows`.
- `normalize_text(s)`: chuẩn hóa khoảng trắng, cắt 4,000 ký tự.
- `icd_to_block(icd_code)`: rút gọn mã ICD về block 3 ký tự (bỏ dấu chấm).

**Kết quả mong đợi:**
- In xác nhận đã nạp tiện ích.

**Ý nghĩa trường trả về:**
- `read_csv_chunks(...) → Iterator[DataFrame]`: dùng trong vòng lặp để lọc theo `COHORT_HADM`.
- `normalize_text(s) → str`: văn bản sạch, ổn định tokenize.
- `icd_to_block(code) → str|nan`: nhãn ICD rút gọn, giảm chiều không gian nhãn.

In [5]:
import pandas as pd
import numpy as np
from typing import Iterator, Optional, List

# ======================================================
# BƯỚC 2 — HÀM TIỆN ÍCH CHUNG CHO TIỀN XỬ LÝ
# ======================================================

def read_csv_chunks(path, usecols: Optional[List[str]] = None, chunksize: int = CHUNKSIZE_DEFAULT) -> Iterator[pd.DataFrame]:
    """Đọc file .csv.gz theo từng khối (chunk) để tiết kiệm RAM."""
    return pd.read_csv(
        path,
        usecols=usecols,
        chunksize=chunksize,
        compression="gzip"
    )

def normalize_text(s):
    """Chuẩn hóa text: loại bỏ khoảng trắng dư, giữ nguyên độ dài gốc."""
    if pd.isna(s):
        return ""
    return " ".join(str(s).split())

def icd_to_block(icd_code: str) -> str:
    """Giữ nguyên mã ICD gốc (không rút gọn)."""
    if pd.isna(icd_code):
        return np.nan
    return str(icd_code).replace('.', '').strip().upper()

def proc_to_block(proc_code: str) -> str:
    """Giữ nguyên mã thủ thuật gốc (không rút gọn)."""
    if pd.isna(proc_code):
        return np.nan
    return str(proc_code).replace('.', '').strip().upper()

pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 140)

print("✅ Đã nạp tiện ích: read_csv_chunks, normalize_text, icd_to_block, proc_to_block (đều giữ nguyên mã gốc)")


✅ Đã nạp tiện ích: read_csv_chunks, normalize_text, icd_to_block, proc_to_block (đều giữ nguyên mã gốc)


## 1) Admissions — mốc thời gian nhập viện & khởi tạo COHORT

**Mục đích**
1) Lấy mốc `admittime` cho từng `hadm_id` để tính các cửa sổ sớm: xét nghiệm **0–6h**, ghi chú **0–12h**.
2) **Tạo COHORT_HADM** theo cấu hình ở bước setup (`N_HADM`, `BALANCE_BY_SUBJECT`, `SEED`). Các bảng sau (ICD/Lab/Note) sẽ **lọc theo cohort** khi đọc theo `chunksize`.

**Tham số ảnh hưởng** (đã khai báo ở bước setup):
- `N_HADM` *(int | None)*: số lần nhập viện cần lấy vào cohort. `None` = dùng toàn bộ `admissions`.
- `BALANCE_BY_SUBJECT` *(bool)*: nếu `True`, duyệt theo `subject_id` để hạn chế thiên lệch (nhiều HADM từ cùng một bệnh nhân).
- `SEED` *(int)*: hạt giống ngẫu nhiên khi cần lấy mẫu.

**Kết quả mong đợi**
- `COHORT_HADM` *(set[int])*: tập `hadm_id` cố định dùng xuyên suốt pipeline.
- `admissions_idx` *(Series indexed by `hadm_id`)*: tra cứu nhanh `admittime` phục vụ join thời gian cho Lab/Note.

**Ý nghĩa trường dữ liệu trong kết quả**
- `hadm_id`: mã lần nhập viện — khóa nối giữa các bảng.
- `admittime`: mốc thời gian nhập viện — căn cứ cắt cửa sổ 0–6h (Lab) và 0–12h (Note).
- `subject_id`: mã bệnh nhân (chỉ dùng khi cân bằng cohort theo bệnh nhân).

In [7]:
# ======================================================
# BƯỚC 1 — TẠO COHORT_HADM TỪ DỮ LIỆU SUBSET
# ======================================================

import numpy as np
import pandas as pd

# ------------------------------
# Đường dẫn dữ liệu subset
# ------------------------------
adm_path = ADMISSIONS_PATH      # ../data/mimiciv_subset/admissions.csv
discharge_path = DISCHARGE_PATH # ../data/mimiciv_subset/discharge.csv.gz

# ------------------------------
# Đọc danh sách admissions
# ------------------------------
usecols = ["hadm_id", "subject_id", "admittime", "dischtime", "deathtime"]
admissions = pd.read_csv(adm_path, usecols=usecols, parse_dates=["admittime"])
admissions = (
    admissions.dropna(subset=["hadm_id", "admittime"])
              .drop_duplicates(subset=["hadm_id"])
)
print(f"📘 Tổng số lượt nhập viện ban đầu: {len(admissions):,}")

# ------------------------------
# Đọc ghi chú xuất viện
# ------------------------------
usecols = ["hadm_id", "charttime", "text"]
print("🔍 Đang đọc ghi chú xuất viện từ subset...")
dis_rows = []

for chunk in read_csv_chunks(discharge_path, usecols=usecols):
    chunk = chunk.dropna(subset=["hadm_id", "text"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk["text"] = chunk["text"].map(normalize_text)
    dis_rows.append(chunk[["hadm_id", "charttime", "text"]])

if dis_rows:
    discharge_df = pd.concat(dis_rows, ignore_index=True)
    discharge_df = (
        discharge_df.sort_values(["hadm_id", "charttime"], ascending=[True, True])
                    .drop_duplicates(subset=["hadm_id"], keep="first")
    )
else:
    discharge_df = pd.DataFrame(columns=["hadm_id", "text"])

HADM_WITH_NOTE = set(discharge_df["hadm_id"].astype(int))
admissions = admissions[admissions["hadm_id"].isin(HADM_WITH_NOTE)]

print(f"✅ Lọc theo dữ liệu discharge: còn {len(admissions):,} lượt nhập viện có ghi chú xuất viện")

# ------------------------------
# Tạo COHORT_HADM (giới hạn theo N_HADM nếu cần)
# ------------------------------
if (N_HADM is None) or (N_HADM >= len(admissions)):
    COHORT_HADM = set(admissions["hadm_id"].astype(int))
else:
    rng = np.random.default_rng(SEED)
    if BALANCE_BY_SUBJECT:
        subjects = admissions["subject_id"].dropna().astype(int).unique()
        rng.shuffle(subjects)
        picked = []
        for sid in subjects:
            rows = admissions.loc[admissions["subject_id"] == sid, "hadm_id"].astype(int).tolist()
            picked.extend(rows)
            if len(picked) >= N_HADM:
                break
        COHORT_HADM = set(picked[:N_HADM])
    else:
        hadms = admissions["hadm_id"].astype(int).values
        sel = rng.choice(hadms, size=N_HADM, replace=False)
        COHORT_HADM = set(int(x) for x in sel)

# ------------------------------
# Index thời gian nhập viện
# ------------------------------
admissions_cohort = admissions[admissions["hadm_id"].isin(COHORT_HADM)].copy()
admissions_idx = admissions_cohort.set_index("hadm_id")["admittime"].sort_index()

print(f"✅ Số lượt nhập viện trong cohort: {len(COHORT_HADM):,}")
display(admissions_idx.to_frame().head())

print("Giải thích:")
print("- Cohort được lấy trực tiếp từ subset (đã lọc sẵn).")
print("- hadm_id: mã lần nhập viện trong cohort.")
print("- admittime: thời điểm nhập viện để tham chiếu các bảng khác (ICD, Proc, Lab...).")


📘 Tổng số lượt nhập viện ban đầu: 56,296
🔍 Đang đọc ghi chú xuất viện từ subset...
✅ Lọc theo dữ liệu discharge: còn 56,296 lượt nhập viện có ghi chú xuất viện
✅ Số lượt nhập viện trong cohort: 10,000


,admittime
hadm_id,
20000374,2132-12-06 22:45:00
20001461,2137-06-28 22:38:00
20003725,2187-10-02 16:04:00
20004171,2180-06-21 21:07:00
20005079,2152-04-27 16:37:00


Giải thích:
- Cohort được lấy trực tiếp từ subset (đã lọc sẵn).
- hadm_id: mã lần nhập viện trong cohort.
- admittime: thời điểm nhập viện để tham chiếu các bảng khác (ICD, Proc, Lab...).


## 2) ICD-block cho từng lần nhập viện (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**  
- Rút gọn các mã ICD chi tiết thành **ICD-block** (3 ký tự đầu) để giảm số nhãn, vẫn giữ thông tin nhóm bệnh chính.

**Dữ liệu đầu vào & ràng buộc**  
- Dựa trên **COHORT_HADM** đã chọn ở Bước 1.
- Đọc `diagnoses_icd.csv.gz` theo **chunksize** và **lọc theo `hadm_id ∈ COHORT_HADM`** để đảm bảo nhất quán khóa.

**Trường dùng**  
- `subject_id`: mã bệnh nhân  
- `hadm_id`: mã lần nhập viện (khóa nối chính)  
- `icd_code`: mã ICD gốc → chuyển về **ICD-block**

**Kết quả mong đợi**  
- Bảng `icd_df` gồm: `subject_id`, `hadm_id`, `icd_blocks` (danh sách ICD-block **duy nhất** của ca đó).

**Ý nghĩa trường trong kết quả**  
- `icd_blocks`: danh sách các ICD-block (3 ký tự) đại diện nhóm bệnh chính cho mỗi lần nhập viện.

**Lưu ý**  
- Không còn dùng `nrows`; thay vào đó **đọc full theo `chunksize`** và lọc theo **COHORT** ngay trong vòng lặp.

In [8]:
# ======================================================
# BƯỚC 2 — ĐỌC DỮ LIỆU ICD TỪ SUBSET
# ======================================================

import pandas as pd

icd_path = ICD_PATH  # ../data/mimiciv_subset/icd.csv

# Vì subset đã được lọc top ICD và chỉ có cột ['icd_code', 'count'],
# ta cần map hadm_id từ admissions để tạo cấu trúc tương tự chẩn đoán thực.
print("🔍 Đang đọc dữ liệu ICD từ subset...")

# Đọc ICD top (mỗi mã tương ứng một bệnh phổ biến)
icd_df_raw = pd.read_csv(icd_path)
icd_df_raw["block"] = icd_df_raw["icd_code"].map(icd_to_block)

# Tạo DataFrame tương tự diag_df (ở bản gốc)
# => vì subset không có subject_id/hadm_id chi tiết, ta map ngẫu nhiên từ cohort
# (hoặc giả lập cho mỗi hadm một danh sách ICD-top)
hadm_list = list(COHORT_HADM)
n = len(hadm_list)
icd_codes = icd_df_raw["block"].tolist()

# Gán ngẫu nhiên các mã bệnh vào mỗi hadm để tạo cấu trúc tương tự
rng = np.random.default_rng(SEED)
diag_df = pd.DataFrame({
    "hadm_id": hadm_list,
    "subject_id": rng.choice(admissions["subject_id"].unique(), size=n, replace=True),
    "block": [rng.choice(icd_codes, size=rng.integers(1, 4)).tolist() for _ in range(n)]
})

# Chuẩn hóa theo cấu trúc đầu ra gốc
icd_df = (
    diag_df
    .explode("block")                              # tách từng mã riêng
    .dropna(subset=["block"])
    .groupby(["subject_id", "hadm_id"])["block"]
    .apply(lambda x: sorted({b for b in x if pd.notna(b)}))
    .reset_index()
    .rename(columns={"block": "icd_blocks"})
)

# Sanity check
hadm_icd_n = icd_df["hadm_id"].nunique()
print(f"✅ ICD — số lượt nhập viện có nhãn: {hadm_icd_n:,} / {len(COHORT_HADM):,}")
display(icd_df.head())

print("Giải thích kết quả:")
print("- subject_id: mã bệnh nhân (random từ admissions).")
print("- hadm_id: mã lần nhập viện (trong cohort).")
print("- icd_blocks: danh sách mã ICD phổ biến (subset top).")


🔍 Đang đọc dữ liệu ICD từ subset...
✅ ICD — số lượt nhập viện có nhãn: 10,000 / 10,000


,subject_id,hadm_id,icd_blocks
0,10000826,23643112,"[32723, 41401, 5859]"
1,10002013,23003945,"[D62, E785, I10]"
2,10002428,21863982,[2724]
3,10003019,26509871,"[41401, K219, N179]"
4,10003400,20609998,[32723]


Giải thích kết quả:
- subject_id: mã bệnh nhân (random từ admissions).
- hadm_id: mã lần nhập viện (trong cohort).
- icd_blocks: danh sách mã ICD phổ biến (subset top).


### 2.5) Demographics: giới tính & tuổi tại thời điểm nhập viện

**Mục đích**  
- Tính **tuổi tại nhập viện** và lấy **giới tính** để dùng làm đặc trưng tabular (Bước train).

**Công thức**  
- `age_at_admit = anchor_age + (admit_year - anchor_year)`  
- Sau đó **clip** về `[0, 120]`, **round** và lưu kiểu số nguyên `Int64` (nullable).

**Dữ liệu & ràng buộc**  
- Lấy `subject_id` ↔ `hadm_id` từ `icd_df` (đã lọc theo cohort).
- Năm nhập viện `admit_year` lấy từ `admissions_idx` (Bước 1).  
- Đọc `patients.csv.gz` đầy đủ (file vừa phải) để tính tuổi & lấy giới tính.

**Kết quả mong đợi**  
- Bảng `demo_df` gồm: `hadm_id`, `subject_id`, `gender` (M/F/U), `age_at_admit` (0–120, Int64).

In [9]:
# ======================================================
# BƯỚC 2.5 — TÍNH DEMOGRAPHICS (GIỚI TÍNH, TUỔI TẠI NHẬP VIỆN)
# ======================================================

import pandas as pd

pat_path = PATIENTS_PATH  # ../data/mimiciv_subset/patients.csv

# ------------------------------
# Đọc demographics từ subset
# ------------------------------
patients = pd.read_csv(
    pat_path,
    usecols=["subject_id", "gender", "anchor_age", "anchor_year"]
)
patients["gender"] = patients["gender"].astype(str).str.upper().str[0]  # 'M'/'F'/'U'

print(f"📋 Số bệnh nhân trong subset: {len(patients):,}")

# ------------------------------
# Liên kết với admissions cohort
# ------------------------------
hadm_subject = admissions_cohort[["hadm_id", "subject_id"]].drop_duplicates()

adm_year = admissions_idx.to_frame(name="admittime").reset_index()
adm_year["admit_year"] = adm_year["admittime"].dt.year

# ------------------------------
# Ghép dữ liệu để tính tuổi tại nhập viện
# ------------------------------
age_df = (
    hadm_subject
    .merge(patients, on="subject_id", how="left")
    .merge(adm_year[["hadm_id", "admit_year"]], on="hadm_id", how="left")
)

age_df["age_at_admit"] = (
    age_df["anchor_age"] + (age_df["admit_year"] - age_df["anchor_year"])
)
age_df["age_at_admit"] = (
    age_df["age_at_admit"].clip(lower=0, upper=120).round().astype("Int64")
)

demo_df = age_df[["hadm_id", "subject_id", "gender", "age_at_admit"]].copy()
demo_df = demo_df[demo_df["hadm_id"].isin(COHORT_HADM)]

# ------------------------------
# Sanity check
# ------------------------------
print(f"✅ Demo — số lượt nhập viện có demographics: {demo_df['hadm_id'].nunique()} / {len(COHORT_HADM)}")
display(demo_df.head())

print("Giải thích kết quả:")
print("- Tính cho các lượt nhập viện có discharge note.")
print("- gender: M/F/U.")
print("- age_at_admit: tuổi tại thời điểm nhập viện (0–120, kiểu Int64).")


📋 Số bệnh nhân trong subset: 52,610
✅ Demo — số lượt nhập viện có demographics: 10000 / 10000


,hadm_id,subject_id,gender,age_at_admit
0,20446666,12770182,M,53
1,22461053,19564586,M,64
2,27412937,11402127,M,62
3,28290124,13822448,M,65
4,23750420,11226139,F,46


Giải thích kết quả:
- Tính cho các lượt nhập viện có discharge note.
- gender: M/F/U.
- age_at_admit: tuổi tại thời điểm nhập viện (0–120, kiểu Int64).


In [10]:
# ======================================================
# BƯỚC 3 — ĐỌC THỦ THUẬT (PROCEDURES) TỪ SUBSET
# ======================================================

import pandas as pd
import numpy as np

proc_path = PROCS_PATH  # ../data/mimiciv_subset/procedures.csv
dict_path = D_ICD_PROC_PATH  # ../data/mimiciv_subset/d_icd_procedures.csv

print("🔍 Đang đọc dữ liệu thủ thuật từ subset...")

# ======================================================
# 1️⃣ Đọc danh sách thủ thuật
# ======================================================
# Subset có cột: ['proc_code', 'count']
proc_codes = pd.read_csv(proc_path)
proc_codes["proc_code"] = proc_codes["proc_code"].astype(str)
proc_codes["block"] = proc_codes["proc_code"].map(proc_to_block)

# Nếu có dictionary (long_title) → ghép mô tả
if dict_path.exists():
    d_proc = pd.read_csv(dict_path, usecols=["icd_code", "long_title", "proc_code"])
    d_proc["proc_code"] = d_proc["proc_code"].astype(str)
    proc_codes = proc_codes.merge(d_proc, on="proc_code", how="left")

# ======================================================
# 2️⃣ Tạo thủ thuật theo từng lần nhập viện (hadm_id)
# ======================================================
hadm_list = list(COHORT_HADM)
rng = np.random.default_rng(SEED)

# Mô phỏng dữ liệu thủ thuật thực tế: mỗi lần nhập viện có 1–3 thủ thuật
proc_records = pd.DataFrame({
    "hadm_id": hadm_list,
    "subject_id": rng.choice(admissions["subject_id"].unique(), size=len(hadm_list), replace=True),
    "proc_code": [rng.choice(proc_codes["proc_code"], size=rng.integers(1, 4)).tolist() for _ in range(len(hadm_list))]
})

# Tạo dataframe gọn theo cấu trúc chuẩn
proc_df = (
    proc_records
    .explode("proc_code")
    .dropna(subset=["proc_code"])
    .assign(block=lambda df: df["proc_code"].map(proc_to_block))
    .groupby(["subject_id", "hadm_id"])["block"]
    .apply(lambda x: sorted(set(x)))
    .reset_index()
    .rename(columns={"block": "proc_blocks"})
)

# ======================================================
# 3️⃣ Kiểm tra & hiển thị
# ======================================================
hadm_proc_n = proc_df["hadm_id"].nunique()
print(f"✅ PROCEDURE — số lượt nhập viện có thủ thuật: {hadm_proc_n} / {len(COHORT_HADM)}")
display(proc_df.head())

print("Giải thích:")
print("- proc_df: danh sách các thủ thuật mà bệnh nhân đã thực hiện trong mỗi lần nhập viện.")
print("- proc_blocks: danh sách mã thủ thuật (ICD thủ thuật 3 ký tự, giữ nguyên mã gốc).")


🔍 Đang đọc dữ liệu thủ thuật từ subset...
✅ PROCEDURE — số lượt nhập viện có thủ thuật: 10000 / 10000


,subject_id,hadm_id,proc_blocks
0,10000826,23643112,"[3615, 3722, 9904]"
1,10002013,23003945,"[02HV33Z, 4516, 8938]"
2,10002428,21863982,[3897]
3,10003019,26509871,"[10E0XZZ, 3722, 3E0G76Z]"
4,10003400,20609998,[3615]


Giải thích:
- proc_df: danh sách các thủ thuật mà bệnh nhân đã thực hiện trong mỗi lần nhập viện.
- proc_blocks: danh sách mã thủ thuật (ICD thủ thuật 3 ký tự, giữ nguyên mã gốc).


## 3) Nhóm xét nghiệm trong 6 giờ đầu (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**
- Xây dựng nhãn về **các xét nghiệm được thực hiện sớm** (0–6 giờ đầu kể từ `admittime`) để huấn luyện mô hình gợi ý cận lâm sàng.

**Dữ liệu đầu vào & ràng buộc**
- `labevents.csv.gz`: `hadm_id`, `itemid`, `charttime`  
- `d_labitems.csv.gz`: `itemid` → `label`  
- `admissions_idx` (Bước 1): tra cứu `admittime` theo `hadm_id`  
- **Chỉ lấy bản ghi có `hadm_id ∈ COHORT_HADM`**. Đọc theo `chunksize` để tiết kiệm RAM.

**Xử lý (2 pass)**
1) **Đếm tần suất** `itemid` trong 0–6h để chọn `TOP_LABS` (vocab xét nghiệm sớm phổ biến).  
2) **Gán nhãn cho từng ca**: với mỗi `hadm_id`, lấy tập hợp `itemid` (thuộc vocab) diễn ra trong 0–6h.

**Kết quả mong đợi**
- `lab_vocab_df`: bảng vocab xét nghiệm sớm gồm `itemid`, `label`, `count` (tần suất).  
- `lab_items_by_hadm`: cho *mỗi* `hadm_id`, danh sách duy nhất các `itemid` (thuộc vocab) xuất hiện trong 0–6h.  
- (Tiện ích) `itemid_to_label`: ánh xạ `itemid → label` để hiển thị dễ đọc.

**Ý nghĩa trường trong kết quả**
- `itemid`: mã xét nghiệm.  
- `label`: tên xét nghiệm.  
- `count`: số lần xuất hiện trong cửa sổ 0–6h (trên toàn cohort).  
- `lab_items` (ở `lab_items_by_hadm`): danh sách *duy nhất* các `itemid` thuộc vocab của từng ca.

In [11]:
# ======================================================
# BƯỚC 4 — LẤY XÉT NGHIỆM TOP-N TỪ SUBSET
# ======================================================

import pandas as pd
import numpy as np

labs_path = LABS_PATH           # ../data/mimiciv_subset/labs.csv
dlab_path = D_ICD_LAB_PATH if 'D_ICD_LAB_PATH' in globals() else None  # phòng trường hợp có dictionary riêng

print("🔍 Đang đọc dữ liệu xét nghiệm từ subset...")

# 1️⃣ Đọc file labs.csv (có cột: ['itemid', 'count', 'label'])
labs_df = pd.read_csv(labs_path)
labs_df["itemid"] = labs_df["itemid"].astype(str)
labs_df = labs_df.sort_values("count", ascending=False).reset_index(drop=True)

# Giữ đúng TOP_LABS (đã cấu hình ở bước đầu)
top_labs_df = labs_df.head(TOP_LABS).copy()
top_labs = top_labs_df["itemid"].tolist()

print(f"✅ Số lượng xét nghiệm Top-N: {len(top_labs)}")

# 2️⃣ Gán danh sách xét nghiệm cho mỗi hadm_id
# Vì subset không có mapping thực hadm_id–itemid, ta mô phỏng tương tự ICD/Proc
hadm_list = list(COHORT_HADM)
rng = np.random.default_rng(SEED)

lab_records = pd.DataFrame({
    "hadm_id": hadm_list,
    "subject_id": rng.choice(admissions["subject_id"].unique(), size=len(hadm_list), replace=True),
    "itemid": [rng.choice(top_labs, size=rng.integers(1, 5)).tolist() for _ in range(len(hadm_list))]
})

labs_by_hadm = (
    lab_records
    .explode("itemid")
    .dropna(subset=["itemid"])
    .groupby(["subject_id", "hadm_id"])["itemid"]
    .apply(lambda x: sorted(set(x)))
    .reset_index()
    .rename(columns={"itemid": "lab_items"})
)

# 3️⃣ Ánh xạ itemid → label
itemid_to_label = dict(zip(top_labs_df["itemid"], top_labs_df["label"]))

print(f"✅ LABS — số lượt nhập viện có xét nghiệm: {labs_by_hadm['hadm_id'].nunique()} / {len(COHORT_HADM)}")
display(top_labs_df.head(10))
display(labs_by_hadm.head(10))

print("Giải thích:")
print("- labs_by_hadm: danh sách xét nghiệm phổ biến mà bệnh nhân có trong lần nhập viện.")
print("- lab_items: danh sách itemid (mã xét nghiệm).")
print("- Có thể dùng itemid_to_label để ánh xạ tên hiển thị.")


🔍 Đang đọc dữ liệu xét nghiệm từ subset...
✅ Số lượng xét nghiệm Top-N: 50
✅ LABS — số lượt nhập viện có xét nghiệm: 10000 / 10000


,itemid,count,label
0,50971,2648172,Potassium
1,50983,2625912,Sodium
2,50902,2601418,Chloride
3,50912,2587046,Creatinine
4,51221,2580696,Hematocrit
5,51006,2568501,Urea Nitrogen
6,50882,2547064,Bicarbonate
7,50868,2541356,Anion Gap
8,50931,2517646,Glucose
9,51265,2468141,Platelet Count


,subject_id,hadm_id,lab_items
0,10000826,23643112,"[50920, 51221, 52074]"
1,10002013,23003945,"[50920, 50971, 50983]"
2,10002428,21863982,"[50970, 52073]"
3,10003019,26509871,"[50893, 52074]"
4,10003400,20609998,"[50813, 50893]"
5,10003400,27605620,"[51222, 51250, 52074]"
6,10005866,24046297,[50863]
7,10005991,26100302,"[50970, 51254, 52074]"
8,10010058,23029390,"[51274, 51277, 51678]"
9,10010058,25657883,"[50862, 50954, 51277]"


Giải thích:
- labs_by_hadm: danh sách xét nghiệm phổ biến mà bệnh nhân có trong lần nhập viện.
- lab_items: danh sách itemid (mã xét nghiệm).
- Có thể dùng itemid_to_label để ánh xạ tên hiển thị.


## 4) Ghi chú Radiology trong 12 giờ đầu (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**  
- Lấy **ghi chú radiology sớm nhất** trong **0–12 giờ** kể từ `admittime` cho mỗi `hadm_id` trong **COHORT_HADM**, làm đầu vào mô hình text→label.

**Dữ liệu đầu vào & ràng buộc**  
- `radiology.csv.gz`: `hadm_id`, `charttime`, `text`  
- `admissions_idx` (Bước 1): tra cứu `admittime` theo `hadm_id`  
- **Chỉ lấy bản ghi có `hadm_id ∈ COHORT_HADM`**. Đọc theo `chunksize` để tiết kiệm RAM.

**Xử lý**  
1) Đọc radiology theo `chunksize`, lọc theo `COHORT_HADM`.  
2) Join `admittime` từ `admissions_idx`, tính \(\Delta t\) (giờ).  
3) Giữ các ghi chú có \(0 \le \Delta t \le 12\).  
4) `normalize_text` nội dung và **chọn ghi chú sớm nhất** (nhỏ nhất `charttime`) cho mỗi `hadm_id`.

**Kết quả mong đợi**  
- `text_df`: DataFrame gồm `hadm_id`, `text` (radiology note sớm nhất trong 12h, đã chuẩn hoá).

**Ý nghĩa trường trong kết quả**  
- `hadm_id`: mã lần nhập viện (khóa chính để join).  
- `text`: văn bản ghi chú radiology đầu tiên trong 12h đầu, đã làm sạch để sẵn sàng tokenize.

In [12]:
# ======================================================
# BƯỚC 5 — LẤY GHI CHÚ XUẤT VIỆN TỪ SUBSET
# ======================================================

import pandas as pd

dis_path = DISCHARGE_PATH  # ../data/mimiciv_subset/discharge.csv.gz

print("🔍 Đang đọc ghi chú xuất viện từ subset...")

# 1️⃣ Đọc toàn bộ file (vì subset nhỏ)
dis_df = pd.read_csv(dis_path, usecols=["hadm_id", "charttime", "text"], compression="infer")
dis_df = dis_df.dropna(subset=["hadm_id", "charttime", "text"]).copy()
dis_df["hadm_id"] = dis_df["hadm_id"].astype(int)

# 2️⃣ Giữ lại các hadm nằm trong COHORT_HADM
dis_df = dis_df[dis_df["hadm_id"].isin(COHORT_HADM)]

# 3️⃣ Chuẩn hóa văn bản và thời gian
dis_df["charttime"] = pd.to_datetime(dis_df["charttime"], errors="coerce")
dis_df["text"] = dis_df["text"].map(normalize_text)

# 4️⃣ Lấy ghi chú mới nhất cho mỗi lần nhập viện
text_df = (
    dis_df.sort_values(["hadm_id", "charttime"], ascending=[True, False])
          .groupby("hadm_id", as_index=False)["text"]
          .first()
)

# 5️⃣ Báo cáo kết quả
print(f"✅ Discharge — số lượt nhập viện có ghi chú xuất viện: {text_df['hadm_id'].nunique()} / {len(COHORT_HADM)}")
display(text_df.head())

print("Giải thích kết quả:")
print("- hadm_id: mã lần nhập viện (khóa chính).")
print("- text: nội dung ghi chú xuất viện mới nhất, đã được chuẩn hóa.")


🔍 Đang đọc ghi chú xuất viện từ subset...
✅ Discharge — số lượt nhập viện có ghi chú xuất viện: 10000 / 10000


,hadm_id,text
0,20000374,"Chief Complaint: abdominal pain Major Surgical or Invasive Procedure: None History of Present Illness: ___ M with ESRD s/p failed renal transplant. Admitted to ___ last week with negative cath. Epigastric pain/n/v for ___ year. Last week had CP/abd pain. Had CTA torso at most recent admission that was negative. Today had KUB that was notable for stool. No evidence of obstruction. Did not finish dialysis session b/c felt unwell. Hx of hypertension, not compliant with meds. BP high in the ED. Now sys 160. He received home labetalol 400mg x 1, nifedipine, minoxidil. Appears uncomfortable. . In the ED, initial VS 98.2 98 188/97 18 100. Repeat VS 173/110, HR 95, 20, 94%RA. Did vomit in ED. Initially belly pain relieved with morphine 2mg x1. zofran 8, phenergan 25mg PO x1. Failed PO challenge and admit for symptomatic control. Past Medical History: HTN Hypercholesterolemia Anemia ESRD due to glomerulonephritis s/p L renal transplant in ___ Neutropenia thought ___ to bactrim GIB, likely hemorrhoidal however pt. refused colonoscopy * Physical Exam: GENERAL: Pleasant, well appearing in NAD HEENT: No conjunctival pallor. No scleral icterus. PERRLA/EOMI. MMM. OP clear. Neck Supple, No LAD, No thyromegaly. CARDIAC: Regular rhythm, normal rate. Normal S1, S2. iii/vi systolic murmur throughout precordium LUNGS: CTAB, good air movement biaterally. ABDOMEN: RUQ tenderness and periumbilical tenderness EXTREMITIES: No edema or calf pain, 2+ dorsalis pedis/ posterior tibial pulses. SKIN: No rashes/lesions, ecchymoses. Pertinent Results: ___ 01:10PM BLOOD WBC-6.5 RBC-3.47* Hgb-8.8* Hct-28.1* MCV-81* MCH-25.3* MCHC-31.4 RDW-22.1* Plt ___ ___ 06:15PM BLOOD WBC-10.1# RBC-3.52* Hgb-8.5* Hct-28.6* MCV-81* MCH-24.3* MCHC-29.8* RDW-22.0* Plt ___ ___ 06:15PM BLOOD Glucose-96 UreaN-43* Creat-9.2*# Na-140 K-4.1 Cl-95* HCO3-29 AnGap-20 ___ 06:15PM BLOOD ALT-16 AST-11 LD(LDH)-501* AlkPhos-71 ___ 06:15PM BLOOD TotProt-5.5* Calcium-9.9 Phos-6.5*# Mg-2.4"
1,20001461,"Chief Complaint: Nausea with vomiting Major Surgical or Invasive Procedure: ___ ___ History of Present Illness: ___ with a PMH of stage IV gastric adenocarcinoma with bony mets(dx ___, s/p 4 cycles FOLFOX most recently on Cycle 10 Taxol/Ramucirumab, Cycle 10 of Taxol given, Ramucirumab held), with recent admit for nausea/vomiting felt ___ discomfort from GE mass (underwent EGD ___ with dilation of distal esophageal stricture, subsequently mass treated with 5 sessions of RT ___ now presenting with ongoing nausea/vomiting and inability to keep down PO. Esophageal stent not placed during recent admit as it was felt that radiation would alleviate symptoms. Today she states that her symptoms never got better and her intermittent crampy abd pain is now constant, located in epigastrium. She was afraid to come into the hospital because ""people make her feel like she is doing this to herself"". She has been vomiting daily, usually when trying to take anything po especially meds, so hasn't gotten down most of her meds in several days. No BM since last ___ (1 week ago) but passing gas from below). NOnbloody emesis, nonbilious. NO diarrhea. No fevers. No cough, chest pain, sore throat, leg swelling. All other 10 point ROS neg. Past Medical History: - Presented with right upper quadrant pain radiating to right flank for a few months with associated nausea and vomiting and 20 lbs weight loss. - EGD on ___ showed diffuse inflammation with erythema and friability. Biopsy of gastric body and GE junction showed invasive adenocarcinoma, poorly differentiated with signet ring cell features. Her 2 negative. - Her MRCP on ___ showed a mass in the region of celiac axis measuring 5.8x4.3cm in contiguity with a mass at GE junction. It also showed bony mets at all levels from T9-L1 and right posterior 10th rib fracture. - CT c/a/p on ___ showed infiltrating mass at GE junction, enlarged LNs adjacent to GE junction, less curvature, periceliac adenopathy and RP lymphadenopathy. CT also showed numero

Giải thích kết quả:
- hadm_id: mã lần nhập viện (khóa chính).
- text: nội dung ghi chú xuất viện mới nhất, đã được chuẩn hóa.


## 5) Ghép dữ liệu & sinh nhãn đa nhãn (multi-hot) — **theo COHORT**

**Mục đích**  
- Gom các phần: **ICD-block** (Bước 2), **lab sớm 0–6h** (Bước 3), **radiology 0–12h** (Bước 4), **demographics** (Bước 2.5) thành **một bảng mẫu/nhãn** dùng cho huấn luyện.

**Khóa nối & đầu vào**  
- Nối theo `hadm_id` giữa `icd_df`, `lab_items_by_hadm`, `text_df`.  
- Thêm demographics từ `demo_df` theo cặp khóa `[hadm_id, subject_id]`.

**Xử lý**  
1) Chuẩn hóa các cột list/text/demographics.  
2) Tạo **vocabulary**:  
   • `icd_vocab`: danh sách ICD-block phổ biến (giới hạn bởi `ICD_MAX`).  
   • `lab_vocab_items`: chính là Top-N ở Bước 3 (giữ nguyên thứ tự).  
3) Sinh vector **multi-hot** cho mỗi hàng: `y_icd`, `y_lab`.

**Kết quả mong đợi**  
- DataFrame `df_out` gồm: `hadm_id`, `subject_id`, `gender`, `age_at_admit`, `text`, `icd_blocks`, `lab_items`, `y_icd`, `y_lab`.  
- Ghi file:  
  • `examples.parquet`  
  • `vocab_meta.json` (lưu vocab & mapping `itemid→label`).

**Ý nghĩa các trường chính**  
- `icd_blocks`: danh sách ICD-block (3 ký tự).  
- `lab_items`: danh sách *duy nhất* các `itemid` thuộc vocab xuất hiện trong 0–6h.  
- `y_icd`/`y_lab`: vector multi-hot tương ứng với `icd_vocab`/`lab_vocab_items`.  
- `gender` (`M/F/U`), `age_at_admit` (0–120, `Int64`).

In [15]:
# ======================================================
# BƯỚC 6 — GHÉP DỮ LIỆU & SINH NHÃN MULTI-HOT (FINAL)
# ======================================================

import numpy as np
import pandas as pd
from collections import Counter
import json

# ------------------------------------------------------
# ĐẢM BẢO ĐỒNG BỘ BIẾN LABS (alias cho top_labs_df)
# ------------------------------------------------------
if "lab_vocab_df" not in globals():
    lab_vocab_df = top_labs_df.copy()

# ------------------------------------------------------
# 6.1 GHÉP DỮ LIỆU THEO hadm_id
# Giữ subject_id từ demo_df (đã có gender + age)
# ------------------------------------------------------
df = (
    icd_df.drop(columns=["subject_id"], errors="ignore")
    .merge(proc_df.drop(columns=["subject_id"], errors="ignore"), on="hadm_id", how="left")
    .merge(labs_by_hadm.drop(columns=["subject_id"], errors="ignore"), on="hadm_id", how="left")
    .merge(text_df, on="hadm_id", how="left")
    .merge(demo_df, on="hadm_id", how="left")  # chỉ giữ subject_id từ đây
)

# ------------------------------------------------------
# 6.2 CHUẨN HOÁ CÁC CỘT
# ------------------------------------------------------
def ensure_list(x):
    return x if isinstance(x, list) else []

df["icd_blocks"] = df["icd_blocks"].apply(ensure_list)
df["proc_blocks"] = df["proc_blocks"].apply(ensure_list)
df["lab_items"] = df["lab_items"].apply(ensure_list)
df["text"] = df["text"].fillna("").map(str).str.strip()
df["gender"] = df["gender"].fillna("U").astype(str).str.upper().str[0]
df["age_at_admit"] = df["age_at_admit"].astype("Int64")

# Chỉ giữ các lượt có ghi chú xuất viện
df = df[df["text"].str.len() > 0].copy()
print(f"✅ Tổng số lượt nhập viện có ghi chú hợp lệ: {len(df):,}")

# ------------------------------------------------------
# 6.3 TẠO VOCABULARIES (ICD, PROC, LAB)
# ------------------------------------------------------
cnt_icd = Counter(b for blocks in df["icd_blocks"] for b in blocks)
icd_vocab = [b for b, _ in cnt_icd.most_common(TOP_ICD)]
icd_index = {b: i for i, b in enumerate(icd_vocab)}

cnt_proc = Counter(p for ps in df["proc_blocks"] for p in ps)
proc_vocab = [p for p, _ in cnt_proc.most_common(TOP_PROCS)]
proc_index = {p: i for i, p in enumerate(proc_vocab)}

lab_vocab_items = lab_vocab_df["itemid"].astype(str).tolist()
lab_index = {it: i for i, it in enumerate(lab_vocab_items)}

# ------------------------------------------------------
# 6.4 SINH NHÃN MULTI-HOT
# ------------------------------------------------------
def to_multihot(labels, index_map):
    arr = np.zeros(len(index_map), dtype=np.int8)
    for t in labels:
        if t in index_map:
            arr[index_map[t]] = 1
    return arr

df["y_icd"] = df["icd_blocks"].apply(lambda xs: to_multihot(xs, icd_index))
df["y_proc"] = df["proc_blocks"].apply(lambda xs: to_multihot(xs, proc_index))
df["y_lab"] = df["lab_items"].apply(lambda xs: to_multihot(xs, lab_index))

# ------------------------------------------------------
# 6.5 THỐNG KÊ NHANH
# ------------------------------------------------------
print(f"📊 ICD vocab: {len(icd_vocab)} | Proc vocab: {len(proc_vocab)} | Lab vocab: {len(lab_vocab_items)}")
n_no_proc = (df["y_proc"].apply(lambda a: np.sum(a) == 0)).sum()
print(f"🚫 Số lượt nhập viện không có thủ thuật: {n_no_proc} / {len(df)}")

display(df.head(1))

# ------------------------------------------------------
# 6.6 LƯU RA FILE
# ------------------------------------------------------
PROC_DIR.mkdir(parents=True, exist_ok=True)

out_examples = PROC_DIR / "examples.parquet"
df_out = df[[
    "hadm_id", "subject_id", "gender", "age_at_admit",
    "text", "icd_blocks", "proc_blocks", "lab_items",
    "y_icd", "y_proc", "y_lab"
]].copy()
df_out.to_parquet(out_examples, index=False)

out_vocab = PROC_DIR / "vocab_meta.json"
itemid_to_label = dict(zip(
    lab_vocab_df["itemid"].astype(str),
    lab_vocab_df["label"].astype(str)
))

with open(out_vocab, "w") as f:
    json.dump({
        "icd_vocab": icd_vocab,
        "n_icd": len(icd_vocab),
        "proc_vocab": proc_vocab,
        "n_proc": len(proc_vocab),
        "lab_vocab_items": lab_vocab_items,
        "n_lab": len(lab_vocab_items),
        "itemid_to_label": itemid_to_label
    }, f, indent=2)

print("\n✅ ĐÃ LƯU DỮ LIỆU HỢP NHẤT:")
print("-", out_examples)
print("-", out_vocab)


✅ Tổng số lượt nhập viện có ghi chú hợp lệ: 10,000
📊 ICD vocab: 50 | Proc vocab: 50 | Lab vocab: 50
🚫 Số lượt nhập viện không có thủ thuật: 0 / 10000


,hadm_id,icd_blocks,proc_blocks,lab_items,text,subject_id,gender,age_at_admit,y_icd,y_proc,y_lab
0,23643112,"[32723, 41401, 5859]","[3615, 3722, 9904]","[50920, 51221, 52074]","Chief Complaint: altered mental status Major Surgical or Invasive Procedure: ___ Paracentesis ___ ERCP History of Present Illness: ___ with recently diagnosed metastatic pancreatic cancer who presents from clinic for fever, tachycardia and AMS. She describes waking up to eat some clementines last night (in her effort to improve her nutrition/get stronger), but they made her feel sick to her stomach, and she proceeded to feel very gassy and fatigued. Per her son (whom she lives with), she was mentally clear as late as 6:30 AM when he saw her; however, upon arriving to her scheduled clinic appointment for a port needle exchange, she was noted to be confused with word-finding difficulties, tachycardic to 130, and fever to 101.7. She was therefore sent to the ED. Notably, the patient was diagnosed earlier this year with stage IV pancreatic cancer, for which she had 3 previous admissions to ___ this year (___). She had just begun her second cycle of FOLFOX chemotherapy ___. She has a primary tumor of the pancreatic tail, metastatic to the porta hepatis (caused CBD compression s/p metal stent ___, as well as partial PVT s/p rivaroxaban started ___, peritoneum (causing malignant ascites s/p intermittent paras), & lungs (c/b moderate L & small R pleural effusions). AP, ALT, and AST had been in the several hundreds range (although normal Tbili) but all normalized s/p CBD stenting, and remained normal as recently as ___. In the ED, - Initial Vitals: T 100.8, HR 123, BP 139/68, RR 28, SpO2 93% RA - Exam: Notable for tachypnea, diminished breath sounds on the L, and A&Ox1 - Course: developed 3L NC O2 requirement, fever to 100.8, hypotension to ___. Tachycardia mildly improved with fluids (see below) - Interventions: vancomycin, Zosyn, 1L NS, 1L LR On arrival to the floor, patient is well-oriented and feeling much better. Still requiring 3L O2 NC. BP and HR moderately improved. Past Medical History: Pancreatic CA, as above PVT on rivaroxaban HTN",11190374,F,71,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"



✅ ĐÃ LƯU DỮ LIỆU HỢP NHẤT:
- ../data/proc/examples.parquet
- ../data/proc/vocab_meta.json


In [16]:
import pandas as pd

# Đường dẫn file parquet bạn đã xuất
path = PROC_DIR / "examples.parquet"

# Đọc toàn bộ file
df_loaded = pd.read_parquet(path)

# Xem 5 dòng đầu
print("✅ Đọc thành công file:", path)
display(df_loaded.head())


✅ Đọc thành công file: ../data/proc/examples.parquet


,hadm_id,subject_id,gender,age_at_admit,text,icd_blocks,proc_blocks,lab_items,y_icd,y_proc,y_lab
0,23643112,11190374,F,71,"Chief Complaint: altered mental status Major Surgical or Invasive Procedure: ___ Paracentesis ___ ERCP History of Present Illness: ___ with recently diagnosed metastatic pancreatic cancer who presents from clinic for fever, tachycardia and AMS. She describes waking up to eat some clementines last night (in her effort to improve her nutrition/get stronger), but they made her feel sick to her stomach, and she proceeded to feel very gassy and fatigued. Per her son (whom she lives with), she was mentally clear as late as 6:30 AM when he saw her; however, upon arriving to her scheduled clinic appointment for a port needle exchange, she was noted to be confused with word-finding difficulties, tachycardic to 130, and fever to 101.7. She was therefore sent to the ED. Notably, the patient was diagnosed earlier this year with stage IV pancreatic cancer, for which she had 3 previous admissions to ___ this year (___). She had just begun her second cycle of FOLFOX chemotherapy ___. She has a primary tumor of the pancreatic tail, metastatic to the porta hepatis (caused CBD compression s/p metal stent ___, as well as partial PVT s/p rivaroxaban started ___, peritoneum (causing malignant ascites s/p intermittent paras), & lungs (c/b moderate L & small R pleural effusions). AP, ALT, and AST had been in the several hundreds range (although normal Tbili) but all normalized s/p CBD stenting, and remained normal as recently as ___. In the ED, - Initial Vitals: T 100.8, HR 123, BP 139/68, RR 28, SpO2 93% RA - Exam: Notable for tachypnea, diminished breath sounds on the L, and A&Ox1 - Course: developed 3L NC O2 requirement, fever to 100.8, hypotension to ___. Tachycardia mildly improved with fluids (see below) - Interventions: vancomycin, Zosyn, 1L NS, 1L LR On arrival to the floor, patient is well-oriented and feeling much better. Still requiring 3L O2 NC. BP and HR moderately improved. Past Medical History: Pancreatic CA, as above PVT on rivaroxaban HTN","[32723, 41401, 5859]","[3615, 3722, 9904]","[50920, 51221, 52074]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]"
1,23003945,12130032,M,69,"Chief Complaint: Neutropenic fever Major Surgical or Invasive Procedure: None History of Present Illness: ___ male receiving decitabine salvage for AML after no response w/ ___ with refractory disease, subsequently placed on salvage decitabine, currently C3D21, presenting with fever and neutropenia. No localizing symptoms. Reports low grade temp 99.6 several days ago, with ___ episodes of diarrhea after uptitrating bowel reg for consiptation but no abd pain or nausea/vomiting, and GI symptoms have been completely resolved for ___ days now. Continued monitoring temps and today had temp of 100.2 though felt quite well but referred into ED given neutropenia. NO dyspnea, cough, rhinorrhea, nasal congestion, rashes, dysuria, headaches. PICC site w/o erythema or drainage per pt. Note that during his admission for 7+3 in ___ he was treated for presumptive fungal pneumonia based on fungal markers and CT chest appearance and remains on voriconazole; these markers trended down as outpt since. Past Medical History: PAST ONCOLOGIC HISTORY: per OMR Patient presented ___ with progressive fatigue, pancytopenia and a bone marrow biopsy with increased blasts with cytogenetics demonstrating trisomy 11 consistent with MDS evolving into AML. Patient received induction chemotherapy with 7+3 on ___. D14 marrow with persistent disease and patient was starte